<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

# Filter Operators with Pangeo

<img src="https://raw.githubusercontent.com/pangeo-data/pangeo.io/refs/heads/main/public/Pangeo-assets/pangeo_logo.png"
     alt="Pangeo logo"
     style="float: center; margin-right: 10px;" />

## Filter Operators

When interacting with large data collections, it is necessary to keep in mind that it's not possible to load everything!

Therefore, we always have to define our requirements in advance and apply them to the data using filter operators.

Let's start again with the same sample data from the Sentinel-2 STAC Collection with an additional filter.

### Properties Filter

When working with optical data like Sentinel-2, most of the times we would like to discard cloudy acquisitions as soon as possible.

We can do it using a property filter: in this case we want to keep only the acquisitions with less than 50% cloud coverage.

In [1]:
# STAC Catalogue Libraries
import pystac_client
import stackstac
import numpy as np
from pyproj import Proj

In [2]:
spatial_extent = [11.1, 46.1, 11.5, 46.5]

In [3]:
%%time
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)
items = catalog.search(
    bbox=spatial_extent,
    collections=["sentinel-2-l2a"]
).item_collection()

datacube = stackstac.stack(items,
                     bounds_latlon=spatial_extent,
)
datacube

CPU times: user 6.62 s, sys: 303 ms, total: 6.92 s
Wall time: 2min 25s


/home/conda/cubes-and-clouds/30d8bce6e9d2d3312ad568684b7851b268e5321683bef302b527561c66bcbe23-20240905-143316-989438-487-pangeo/lib/python3.11/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-462e21a55c53dd1627ad3e7b5cb1ddfa' (time: 1397,
                                                                band: 32,
                                                                y: 4535, x: 3210)> Size: 5TB
dask.array<fetch_raster_window, shape=(1397, 32, 4535, 3210), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 11kB 2016-...
    id                                       (time) <U24 134kB 'S2A_32TPS_201...
  * band                                     (band) <U12 2kB 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 26kB 6.611e+05 ... 6...
  * y                                        (y) float64 36kB 5.153e+06 ... 5...
    constellation                            <U10 40B 'sentinel-2'
    ...                                       ...
    raster:bands                             (band) object 256B [{'nodata': 0...
    gsd                                      (band) object 256B None 10 ... None
    common_name                              (band) object 256B None ... None
    center_wavelength                        (band) object 256B None ... None
    full_width_half_max                      (band) object 256B None ... None
    epsg                                     int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(661130.0, 5107300.0, 693230.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 661130.00|\n| 0.00,-10.00, 5152650.00|\n| 0.0...
    resolution:  10.0

Let's filter out scenes with >50% cloud coverage (according to the eo:cloud_cover field set by the data provider).

In [4]:
lowcloud = datacube[datacube["eo:cloud_cover"] < 50]
lowcloud

<xarray.DataArray 'stackstac-462e21a55c53dd1627ad3e7b5cb1ddfa' (time: 757,
                                                                band: 32,
                                                                y: 4535, x: 3210)> Size: 3TB
dask.array<getitem, shape=(757, 32, 4535, 3210), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 6kB 2016-1...
    id                                       (time) <U24 73kB 'S2A_32TPS_2016...
  * band                                     (band) <U12 2kB 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 26kB 6.611e+05 ... 6...
  * y                                        (y) float64 36kB 5.153e+06 ... 5...
    constellation                            <U10 40B 'sentinel-2'
    ...                                       ...
    raster:bands                             (band) object 256B [{'nodata': 0...
    gsd                                      (band) object 256B None 10 ... None
    common_name                              (band) object 256B None ... None
    center_wavelength                        (band) object 256B None ... None
    full_width_half_max                      (band) object 256B None ... None
    epsg                                     int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(661130.0, 5107300.0, 693230.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 661130.00|\n| 0.00,-10.00, 5152650.00|\n| 0.0...
    resolution:  10.0

### Temporal filter

To filter along time the data collection with Pangeo Xarray, we can use `where` method from Xarray. 

**Note**: We cannot use the sel method to select a slice to filter on the time dimension since the times are not equally spaced.

In [5]:
temporal_slice = lowcloud.where((lowcloud.time >= np.datetime64("2022-05-10T00:00:00")) &  (lowcloud.time  <= np.datetime64("2022-06-30T00:00:00")))
temporal_slice

<xarray.DataArray 'stackstac-462e21a55c53dd1627ad3e7b5cb1ddfa' (time: 757,
                                                                band: 32,
                                                                y: 4535, x: 3210)> Size: 3TB
dask.array<where, shape=(757, 32, 4535, 3210), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 6kB 2016-1...
    id                                       (time) <U24 73kB 'S2A_32TPS_2016...
  * band                                     (band) <U12 2kB 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 26kB 6.611e+05 ... 6...
  * y                                        (y) float64 36kB 5.153e+06 ... 5...
    constellation                            <U10 40B 'sentinel-2'
    ...                                       ...
    raster:bands                             (band) object 256B [{'nodata': 0...
    gsd                                      (band) object 256B None 10 ... None
    common_name                              (band) object 256B None ... None
    center_wavelength                        (band) object 256B None ... None
    full_width_half_max                      (band) object 256B None ... None
    epsg                                     int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(661130.0, 5107300.0, 693230.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 661130.00|\n| 0.00,-10.00, 5152650.00|\n| 0.0...
    resolution:  10.0

After running the previous cell, it is visible that the result has less elements (or labels) in the temporal dimension `time`.

Additionally, the size of the selected data reduced a lot.

**Quiz hint: look carefully at the dimensions of the resulting datacube!**

### Spatial filter

To slice along the spatial dimensions the data collection with Pangeo, we can use `sel` method with a `slice`.

Let's get the spatial extent expressed in the Coordinate Reference System of the datacube. We need to project latitudes, longitudes to the data cube CRS.

In [6]:
west = 11.259613; east = 11.406212
south = 46.461019; north = 46.522237
projection = Proj(temporal_slice.attrs["crs"])
x_ws, y_ws = projection(west, south)
x_wn, y_wn = projection(west, north)
x_es, y_es = projection(east, south)
x_en, y_en = projection(east, north)
xmax = max(x_ws, x_wn, x_es, x_en)
xmin = min(x_ws, x_wn, x_es, x_en)
ymax = max(y_ws, y_wn, y_es, y_en)
ymin = min(y_ws, y_wn, y_es, y_en)
print(xmin, xmax, ymin, ymax)

673311.3443826602 684762.3945897217 5147752.717505932 5154887.167103742


The `sel` method with `slice` is used with a set of coordinates. 

**Note**: The order of the interval in the slice needs to be expressed in the same order as the corresponding coordinate.

In [7]:
spatial_slice = temporal_slice.sel(x=slice(xmin,xmax), y = slice(ymax,ymin))
spatial_slice

<xarray.DataArray 'stackstac-462e21a55c53dd1627ad3e7b5cb1ddfa' (time: 757,
                                                                band: 32,
                                                                y: 490, x: 1145)> Size: 109GB
dask.array<getitem, shape=(757, 32, 490, 1145), dtype=float64, chunksize=(1, 1, 490, 829), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 6kB 2016-1...
    id                                       (time) <U24 73kB 'S2A_32TPS_2016...
  * band                                     (band) <U12 2kB 'aot' ... 'wvp-jp2'
  * x                                        (x) float64 9kB 6.733e+05 ... 6....
  * y                                        (y) float64 4kB 5.153e+06 ... 5....
    constellation                            <U10 40B 'sentinel-2'
    ...                                       ...
    raster:bands                             (band) object 256B [{'nodata': 0...
    gsd                                      (band) object 256B None 10 ... None
    common_name                              (band) object 256B None ... None
    center_wavelength                        (band) object 256B None ... None
    full_width_half_max                      (band) object 256B None ... None
    epsg                                     int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(661130.0, 5107300.0, 693230.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 661130.00|\n| 0.00,-10.00, 5152650.00|\n| 0.0...
    resolution:  10.0

**Quiz hint: look carefully at the dimensions of the loaded datacube!**

### Bands filter

To slice along the bands dimension, keeping only the necessary bands, we can use the `sel` method and `isin`.

In [8]:
bands = ["red", "green", "blue"]
bands_slice = spatial_slice.sel(band=spatial_slice.band.isin(bands))
bands_slice

<xarray.DataArray 'stackstac-462e21a55c53dd1627ad3e7b5cb1ddfa' (time: 757,
                                                                band: 3,
                                                                y: 490, x: 1145)> Size: 10GB
dask.array<getitem, shape=(757, 3, 490, 1145), dtype=float64, chunksize=(1, 1, 490, 829), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 6kB 2016-1...
    id                                       (time) <U24 73kB 'S2A_32TPS_2016...
  * band                                     (band) <U12 144B 'blue' ... 'red'
  * x                                        (x) float64 9kB 6.733e+05 ... 6....
  * y                                        (y) float64 4kB 5.153e+06 ... 5....
    constellation                            <U10 40B 'sentinel-2'
    ...                                       ...
    raster:bands                             (band) object 24B None None None
    gsd                                      (band) object 24B 10 10 10
    common_name                              (band) object 24B 'blue' ... 'red'
    center_wavelength                        (band) object 24B 0.49 0.56 0.665
    full_width_half_max                      (band) object 24B 0.098 0.045 0.038
    epsg                                     int64 8B 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(661130.0, 5107300.0, 693230.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 661130.00|\n| 0.00,-10.00, 5152650.00|\n| 0.0...
    resolution:  10.0